# 04 – SQL Integration & Dashboard Queries
**Author:** Mousumi Paul | Feb 2025

Demonstrates connecting to PostgreSQL, running views, and loading data into Python for analysis.

## 1. Setup

In [ ]:
import pandas as pd, sys
sys.path.append('..')
from src.sql_connector import load_from_csv
from src.utils import combined_report, fmt_inr
from src.inventory_engine import run_optimization
from src.alert_engine import generate_alerts
print('✅ Ready')

## 2. Load Data (CSV mode)

In [ ]:
data   = load_from_csv('../data/raw')
params = data['inventory']
sales  = data['sales']
inv_df = run_optimization(params)
alerts = generate_alerts(inv_df)

## 3. Combined Report

In [ ]:
import os; os.makedirs('../outputs/reports', exist_ok=True)
report = combined_report(inv_df, alerts, '../outputs/reports/supply_chain_summary.csv')
report[['Category','EOQ_Units','Safety_Stock','Reorder_Point',
        'Days_of_Supply','Total_Inventory_Cost_INR','Alert_Level','Action_Message']]

## 4. Executive KPI Summary

In [ ]:
from src.alert_engine import alert_summary_stats
stats = alert_summary_stats(inv_df)
total_cost = inv_df['Total_Inventory_Cost_INR'].sum()
total_excess = inv_df['Excess_Holding_Cost_INR'].sum()

print('='*55)
print('📊 SUPPLY CHAIN KPI SUMMARY')
print('='*55)
print(f'  Total Products Monitored:     {stats["total_products"]:>10}')
print(f'  🔴 Critical (below SS):       {stats["critical"]:>10}')
print(f'  🟠 Reorder Now:               {stats["reorder"]:>10}')
print(f'  🟢 Healthy:                   {stats["healthy"]:>10}')
print(f'  Avg Days of Supply:           {stats["avg_days_supply"]:>10.1f} days')
print(f'  Total Annual Inventory Cost:  {fmt_inr(total_cost):>15}')
print(f'  Total Excess Holding Cost:    {fmt_inr(total_excess):>15}')
print('='*55)

## 5. (Optional) Live SQL Connection

In [ ]:
# Uncomment when PostgreSQL is running:
# from src.sql_connector import get_pg_engine, load_inventory_dashboard, load_low_stock_alerts
# engine = get_pg_engine(password='your_password')
# df = load_inventory_dashboard(engine)
# alerts_live = load_low_stock_alerts(engine)
# print_alert_summary(engine)
print('⚠️  Uncomment above when DB is live')

## 6. Simulate vw_kpi_summary SQL Output

In [ ]:
kpi = {
    'total_products':            len(inv_df),
    'critical_count':            (inv_df['Current_Stock'] < inv_df['Safety_Stock']).sum(),
    'reorder_count':             ((inv_df['Current_Stock'] >= inv_df['Safety_Stock']) &
                                  (inv_df['Current_Stock'] <  inv_df['Reorder_Point'])).sum(),
    'healthy_count':             (inv_df['Alert_Status'].str.contains('HEALTHY')).sum(),
    'avg_days_supply':           round(inv_df['Days_of_Supply'].mean(), 1),
    'total_excess_units':        inv_df['Excess_Stock'].sum(),
    'total_excess_holding_cost': inv_df['Excess_Holding_Cost_INR'].sum(),
    'total_annual_inv_cost':     inv_df['Total_Inventory_Cost_INR'].sum(),
}
pd.DataFrame([kpi]).T.rename(columns={0:'Value'})